# XArrayDataset

In [1]:
import autoroot
from oceanbench._src.datasets.base import XRDABatcher
import numpy as np
import pandas as pd
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, 
    TimeAxis, SSH2D, SSH2DT
)
from xarray_dataclasses import asdataarray, asdataset
%load_ext autoreload
%autoreload 2


In [2]:
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, CoordinateAxis,
    TimeAxis, SSH2D, SSH2DT, Bounds
)
from xarray_dataclasses import asdataarray, asdataset, Data, Name, Coordof, Coord
from oceanbench._src.geoprocessing.gridding import create_coord_grid

In [3]:
from dataclasses import dataclass
from typing import Literal, Tuple


X = Literal["x"]
Y = Literal["y"]
Z = Literal["z"]

@dataclass
class Variable1D:
    data: Data[X, np.ndarray]
    x: Coord[X, np.ndarray] = 0
    name: Name[str] = "var"


@dataclass
class Variable2D:
    data: Data[tuple[X,Y], np.ndarray]
    x: Coord[X, np.ndarray] = 0
    y: Coord[Y, np.ndarray] = 0
    name: Name[str] = "var"


@dataclass
class Variable3D:
    data: Data[tuple[X, Y, Z], np.ndarray]
    x: Coord[X, np.ndarray] = 0
    y: Coord[Y, np.ndarray] = 0
    z: Coord[Z, np.ndarray] = 0
    name: Name[str] = "var"



**Cases I**: 1D Time Series

* `[360] * [30p] * [30s] = [12]`
* Latent Vector - `[12, Z]`
* Ensemble Predictions - `[12, 20]`

**Case II**: Lat/Lon

* `[200, 200] * [10, 10] = [20, 20]`

**Case III**: Lat/Lon/Time

* `[200, 200, 360] * [10, 10, 30] = [20, 20, 12]`
* 360 Days to 30 Days
* Latent Vector
* Ensemble Predictions

## 1D Temporal Field

We have a 1D time series:



In [4]:
from oceanbench._src.datasets.base import XRDABatcher

In [5]:

x = CoordinateAxis.init_from_limits(x_min=1, x_max=360, dx = 1)
rng = np.random.RandomState(seed=123)
data = rng.randn(*x.data.shape)

@dataclass
class Variable1D:
    data: Data[X, np.ndarray]
    x: Coord[X, np.ndarray] = 0
    name: Name[str] = "var"
    
variable = Variable1D(data=data, x=x.data, name="ssh")
da = asdataarray(variable)
da

<xarray.DataArray 'ssh' (x: 360)>
array([-1.0856306033005612, 0.9973454465835858, 0.28297849805199204,
       -1.5062947139180922, -0.5786002519685364, 1.651436537097151,
       -2.426679243393074, -0.42891262885617726, 1.2659362587055338,
       -0.8667404022651016, -0.6788861516220543, -0.09470896893689112,
       1.4913896261242878, -0.638901996684651, -0.44398195964606546,
       -0.43435127561851733, 2.205930082725455, 2.1867860889737862,
       1.004053897878877, 0.386186399174856, 0.7373685758962422,
       1.490732028150799, -0.9358338684023914, 1.1758290447821034,
       -1.2538806677490124, -0.6377515024534103, 0.9071051958003014,
       -1.4286807002259692, -0.1400687201886661, -0.8617548958596855,
       -0.2556193705305969, -2.7985891054607244, -1.771533104509847,
       -0.6998772345979173, 0.9274624317585825, -0.1736356827902158,
       0.0028459158968110196, 0.688222711102285, -0.8795363430090519,
       0.283627323807291, -0.8053665180656158, -1.7276694941206072,
       -0.390899793755101, 0.5738058624050577, 0.3385890509998015,
       -0.01183049447881976, 2.392365265937726, 0.4129121603087788,
       0.9787360059373466, 2.2381433384979528, -1.2940853231612488,
       -1.0387882102049535, 1.7437122251229307, -0.7980627352410625,
       0.029683230303330227, 1.0693159694243488, 0.8907063912931706,
       1.754886181981109, 1.4956441370334692, 1.0693926697057368,
...
       0.6111235500559111, -0.14401334748029268, 1.3166055958635523,
       -0.7043421471287583, 0.7506099168684812, 0.3426379813692278,
       -0.12643756370963435, 1.1759107718346071, 0.6800715328674297,
       -1.0049671533821039, 0.6402186804198471, 1.374990631494199,
       -0.13044468898462372, -0.24865585038497331, -0.6696471476435246,
       -0.013603885675351006, 0.68620068605291, -0.8176682995787476,
       -1.34635756073724, -0.3757499109819263, -1.3797249786587495,
       0.5232184412826316, -0.4266897699938011, -1.7554018445508066,
       -0.3486075147385099, -0.1926149857940906, 0.4491356133597172,
       -0.14536354315421035, 1.8687264568398085, -0.5187038515389994,
       -0.06239854858230655, -0.10291061426267058, -0.28262838440702354,
       0.14242558501294, 0.5412312971976727, 1.3400987035364347,
       -1.5692561273944634, -0.5103428738061208, -0.4477714253522432,
       0.937850297361706, -0.3566630610733439, -1.8951755930793188,
       0.0877304638665764, -0.03368923154093914, 0.17975156577131524,
       -1.040162880979482, 1.7190346810845034, -0.32385978378534624,
       -0.18829685849366687, -0.9000085698091608, -0.9310020038747236,
       -1.2227369597487854, -0.3933108517811569, -0.957581838756634,
       2.0564673412252215, -1.8884923803868152, -1.1283305481817938],
      dtype=object)
Coordinates:
  * x        (x) object 1 2 3 4 5 6 7 8 9 ... 353 354 355 356 357 358 359 360

In [32]:
patches = {"x": 60}
strides = {"x": 60}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

# assert ds[0].shape == (len(var.data),)
# np.testing.assert_array_equal(ds[0], np.arange(0, len(var.x), 1))
print(xrda_batches)
print(f"Torch Dataset(size): {len(xrda_batches)}")
# print(f"Batch (size): {ds[0].shape}")

XArray Patcher
DataArray size: OrderedDict([('x', 360)])
Patches:        OrderedDict([('x', 60)])
Strides:        OrderedDict([('x', 60)])
Num Batches:    OrderedDict([('x', 6)])
Torch Dataset(size): 6


In [15]:
import functools as ft
import itertools as it
from einops import repeat

all_batches = list(map(lambda x: x.data, xrda_batches))
all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N", N=5), all_batches)) 

all_coords = xrda_batches.get_coords()

assert len(all_batches) == len(all_coords)

In [35]:
xrda_batches.patches["x"]

60

In [16]:
import itertools

items = list(itertools.chain(*[all_batches]))
items_latent = list(itertools.chain(*[all_batches_latent]))

In [22]:
# create items
all_batches = list(map(lambda x: x.data, xrda_batches))
items = list(itertools.chain(*[all_batches]))
items[0]

array([-1.0856306033005612, 0.9973454465835858, 0.28297849805199204,
       -1.5062947139180922, -0.5786002519685364, 1.651436537097151,
       -2.426679243393074, -0.42891262885617726, 1.2659362587055338,
       -0.8667404022651016, -0.6788861516220543, -0.09470896893689112,
       1.4913896261242878, -0.638901996684651, -0.44398195964606546,
       -0.43435127561851733, 2.205930082725455, 2.1867860889737862,
       1.004053897878877, 0.386186399174856, 0.7373685758962422,
       1.490732028150799, -0.9358338684023914, 1.1758290447821034,
       -1.2538806677490124, -0.6377515024534103, 0.9071051958003014,
       -1.4286807002259692, -0.1400687201886661, -0.8617548958596855],
      dtype=object)

In [26]:
items = list(itertools.chain(all_batches))
items[0]

array([-1.0856306033005612, 0.9973454465835858, 0.28297849805199204,
       -1.5062947139180922, -0.5786002519685364, 1.651436537097151,
       -2.426679243393074, -0.42891262885617726, 1.2659362587055338,
       -0.8667404022651016, -0.6788861516220543, -0.09470896893689112,
       1.4913896261242878, -0.638901996684651, -0.44398195964606546,
       -0.43435127561851733, 2.205930082725455, 2.1867860889737862,
       1.004053897878877, 0.386186399174856, 0.7373685758962422,
       1.490732028150799, -0.9358338684023914, 1.1758290447821034,
       -1.2538806677490124, -0.6377515024534103, 0.9071051958003014,
       -1.4286807002259692, -0.1400687201886661, -0.8617548958596855],
      dtype=object)

In [18]:
# create items
all_batches = list(map(lambda x: x, xrda_batches))
items = list(itertools.chain(*[all_batches]))

# CASE I - Weight Matrix | Explicit Label
dims_labels = ["x"]
weight = np.ones([30,])
rec_da = xrda_batches.reconstruct_from_items(items, dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)

# CASE II - No Weight Matrix | Explicit Label
dims_labels = ["x"]
weight = None
rec_da = xrda_batches.reconstruct_from_items(items, dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)

# CASE III - No Weight Matrix | No Label
dims_labels = None
weight = None
rec_da = xrda_batches.reconstruct_from_items(items, dims_labels=dims_labels, weight=weight)
# rec_da = reconstruct_from_items(items_latent, dims_label_latent, ds, weight=w)

np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)
rec_da



100%|██████████| 67/67 [00:00<00:00, 362.14it/s]


<xarray.DataArray (x: 360)>
array([-1.08563060e+00,  9.97345447e-01,  2.82978498e-01, -1.50629471e+00,
       -5.78600252e-01,  1.65143654e+00, -2.42667924e+00, -4.28912629e-01,
        1.26593626e+00, -8.66740402e-01, -6.78886152e-01, -9.47089689e-02,
        1.49138963e+00, -6.38901997e-01, -4.43981960e-01, -4.34351276e-01,
        2.20593008e+00,  2.18678609e+00,  1.00405390e+00,  3.86186399e-01,
        7.37368576e-01,  1.49073203e+00, -9.35833868e-01,  1.17582904e+00,
       -1.25388067e+00, -6.37751502e-01,  9.07105196e-01, -1.42868070e+00,
       -1.40068720e-01, -8.61754896e-01, -2.55619371e-01, -2.79858911e+00,
       -1.77153310e+00, -6.99877235e-01,  9.27462432e-01, -1.73635683e-01,
        2.84591590e-03,  6.88222711e-01, -8.79536343e-01,  2.83627324e-01,
       -8.05366518e-01, -1.72766949e+00, -3.90899794e-01,  5.73805862e-01,
        3.38589051e-01, -1.18304945e-02,  2.39236527e+00,  4.12912160e-01,
        9.78736006e-01,  2.23814334e+00, -1.29408532e+00, -1.03878821e+00,
        1.74371223e+00, -7.98062735e-01,  2.96832303e-02,  1.06931597e+00,
        8.90706391e-01,  1.75488618e+00,  1.49564414e+00,  1.06939267e+00,
       -7.72708714e-01,  7.94862668e-01,  3.14271995e-01, -1.32626546e+00,
        1.41729905e+00,  8.07236535e-01,  4.54900806e-02, -2.33092061e-01,
       -1.19830114e+00,  1.99524074e-01,  4.68439119e-01, -8.31154984e-01,
        1.16220405e+00, -1.09720305e+00, -2.12310035e+00,  1.03972709e+00,
       -4.03366038e-01, -1.26029585e-01, -8.37516723e-01, -1.60596276e+00,
...
       -1.25118987e+00, -1.18432734e+00, -1.51810798e+00, -4.61187413e-01,
       -3.54908832e-01, -6.82538154e-01, -1.65369784e+00,  1.25333595e+00,
       -1.32907883e+00,  2.78033712e-01, -1.07476659e+00,  6.68316867e-01,
        9.55832355e-01, -8.77613587e-01, -1.92371573e+00,  6.95787319e-01,
        1.87580055e+00,  4.15694540e-01,  1.60544421e-01,  8.19760610e-01,
        7.65054846e-01, -8.28988834e-01, -6.59151311e-01,  6.11123550e-01,
       -1.44013347e-01,  1.31660560e+00, -7.04342147e-01,  7.50609917e-01,
        3.42637981e-01, -1.26437564e-01,  1.17591077e+00,  6.80071533e-01,
       -1.00496715e+00,  6.40218680e-01,  1.37499063e+00, -1.30444689e-01,
       -2.48655850e-01, -6.69647148e-01, -1.36038857e-02,  6.86200686e-01,
       -8.17668300e-01, -1.34635756e+00, -3.75749911e-01, -1.37972498e+00,
        5.23218441e-01, -4.26689770e-01, -1.75540184e+00, -3.48607515e-01,
       -1.92614986e-01,  4.49135613e-01, -1.45363543e-01,  1.86872646e+00,
       -5.18703852e-01, -6.23985486e-02, -1.02910614e-01, -2.82628384e-01,
        1.42425585e-01,  5.41231297e-01,  1.34009870e+00, -1.56925613e+00,
       -5.10342874e-01, -4.47771425e-01,  9.37850297e-01, -3.56663061e-01,
       -1.89517559e+00,  8.77304639e-02, -3.36892315e-02,  1.79751566e-01,
       -1.04016288e+00,  1.71903468e+00, -3.23859784e-01, -1.88296858e-01,
       -9.00008570e-01, -9.31002004e-01, -1.22273696e+00, -3.93310852e-01,
       -9.57581839e-01,  2.05646734e+00, -1.88849238e+00, -1.12833055e+00])
Coordinates:
  * x        (x) object 1 2 3 4 5 6 7 8 9 ... 353 354 355 356 357 358 359 360

In [29]:
tuple(["x", "z"])

('x', 'z')

In [27]:
rec_da.dims

('x',)

In [13]:
from einops import rearrange
all_batches = list(map(lambda x: x, xrda_batches))
all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N", N=5), all_batches)) 
items_latent = list(itertools.chain(*[all_batches_latent]))

# CASE I - Weight  | Label (All)
dims_labels_latent = ["x", "z"]
weight = np.ones([30,])
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da)

# CASE I - Weight  | Label (All) - switched
dims_labels_latent = ["z", "x"]
weight = np.ones([30,])
all_batches_latent_flipped = list(map(lambda x: repeat(x, "D Z -> Z D"), all_batches_latent)) 
items_latent_flipped = list(itertools.chain(*[all_batches_latent_flipped]))
rec_da = xrda_batches.reconstruct_from_items(items_latent_flipped, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da)

# CASE I -  No Weight |  Label (All) - Switched
dims_labels_latent = ["z", "x"]
weight = None
all_batches_latent_flipped = list(map(lambda x: repeat(x, "D Z -> Z D"), all_batches_latent)) 
items_latent_flipped = list(itertools.chain(*[all_batches_latent_flipped]))
rec_da = xrda_batches.reconstruct_from_items(items_latent_flipped, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da)


# CASE I -  No Weight | Half Label (All)
dims_labels_latent = ["x"]
weight = None
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(v1=0).data, xrda_batches.da)

# CASE I -  No Weight | No Label (All)
dims_labels_latent = None
weight = None
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(v1=0).data, xrda_batches.da)

rec_da

100%|██████████| 67/67 [00:00<00:00, 440.82it/s]


<xarray.DataArray (x: 360, v1: 5)>
array([[-1.08563066, -1.08563066, -1.08563066, -1.08563066, -1.08563066],
       [ 0.99734545,  0.99734545,  0.99734545,  0.99734545,  0.99734545],
       [ 0.2829785 ,  0.2829785 ,  0.2829785 ,  0.2829785 ,  0.2829785 ],
       ...,
       [ 2.05646729,  2.05646729,  2.05646729,  2.05646729,  2.05646729],
       [-1.88849235, -1.88849235, -1.88849235, -1.88849235, -1.88849235],
       [-1.12833059, -1.12833059, -1.12833059, -1.12833059, -1.12833059]])
Coordinates:
  * x        (x) object 1 2 3 4 5 6 7 8 9 ... 353 354 355 356 357 358 359 360
Dimensions without coordinates: v1

## Test Case II - 2D Field

In [14]:
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, 
    TimeAxis, SSH2D, SSH2DT
)

In [52]:
rng = np.random.RandomState(123)

lon_axis = LongitudeAxis.init_from_limits(lon_min=10, lon_max=30-0.5, dlon=0.5)
lat_axis = LatitudeAxis.init_from_limits(lat_min=-80, lat_max=-60-0.5, dlat=0.5)

da = SSH2D.init_from_axis(lon=lon_axis, lat=lat_axis)
da.data = rng.randn(*da.data.shape)

da = asdataarray(da)
da

<xarray.DataArray 'ssh' (lat: 40, lon: 40)>
array([[-1.0856307 ,  0.99734545,  0.2829785 , ...,  0.6882227 ,
        -0.87953633,  0.28362733],
       [-0.8053665 , -1.7276695 , -0.3908998 , ..., -0.12602958,
        -0.8375167 , -1.6059628 ],
       [ 1.2552373 , -0.688869  ,  1.6609524 , ..., -0.33247527,
         1.9594114 , -2.0250459 ],
       ...,
       [-0.32313433,  1.165847  ,  0.14398135, ...,  0.5553566 ,
         0.22872074,  0.46431547],
       [-0.30643538, -0.2179235 ,  1.4419992 , ..., -1.1567088 ,
         0.49945688,  0.70969313],
       [-0.17347364, -1.6917511 ,  0.3677055 , ...,  1.152356  ,
        -0.60180914, -1.2156621 ]], dtype=float32)
Coordinates:
  * lat      (lat) float32 -80.0 -79.5 -79.0 -78.5 ... -62.0 -61.5 -61.0 -60.5
  * lon      (lon) float32 10.0 10.5 11.0 11.5 12.0 ... 27.5 28.0 28.5 29.0 29.5
Attributes:
    units:          m
    standard_name:  sea_surface_height
    long_name:      Sea Surface Height

In [76]:
patches = {"lat": 8, "lon": 8}
strides = {"lat": 4, "lon": 4}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

# assert ds[0].shape == (len(var.data),)
# np.testing.assert_array_equal(ds[0], np.arange(0, len(var.x), 1))
print(xrda_batches)
print(f"Torch Dataset(size): {len(xrda_batches)}")
# print(f"Batch (size): {ds[0].shape}")

XArray Patcher
DataArray size: OrderedDict([('lat', 40), ('lon', 40)])
Patches:        OrderedDict([('lat', 8), ('lon', 8)])
Strides:        OrderedDict([('lat', 4), ('lon', 4)])
Num Batches:    OrderedDict([('lat', 9), ('lon', 9)])
Torch Dataset(size): 81


In [63]:
import functools as ft
import itertools as it
from einops import repeat

all_batches = list(map(lambda x: x, xrda_batches))


all_coords = xrda_batches.get_coords()

assert len(all_batches) == len(all_coords)
all_batches[0].shape

(40, 50)

In [75]:
# create items
all_batches = list(map(lambda x: x, xrda_batches))
items = list(itertools.chain(*[all_batches]))

# CASE III - No Weight | No Label
dims_labels = None
weight = None
rec_da = xrda_batches.reconstruct_from_items(items, dims_labels=dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)
rec_da

# CASE II - No Weight | Label
dims_labels = ["lat", "lon"]
weight = None
rec_da = xrda_batches.reconstruct_from_items(items, dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)
rec_da

# CASE I - Weight | Label
dims_labels = ["lat", "lon"]
weight = np.ones([40,50])
rec_da = xrda_batches.reconstruct_from_items(items, dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)
rec_da

# CASE I - Weight | Half Labels
dims_labels = ["lat"]
weight = np.ones([40,])
rec_da = xrda_batches.reconstruct_from_items(items, dims_labels, weight=weight)
rec_da

# CASE - No Weight | Mixed Labels
dims_labels = ["lon", "lat"]
weight = None
all_batches_flipped = list(map(lambda x: repeat(x, "Lat Lon -> Lon Lat"), all_batches)) 
items_flipped = list(itertools.chain(*[all_batches_flipped]))
rec_da = xrda_batches.reconstruct_from_items(items_flipped, dims_labels=dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da.T)
rec_da

# CASE I - Weight | Mixed Half Labels 
dims_labels = ["lon"]
weight = np.ones([50,])
all_batches_flipped = list(map(lambda x: repeat(x, "Lat Lon -> Lon Lat"), all_batches)) 
items_flipped = list(itertools.chain(*[all_batches_flipped]))
rec_da = xrda_batches.reconstruct_from_items(items_flipped, dims_labels, weight=weight)
rec_da



100%|██████████| 144/144 [00:00<00:00, 522.63it/s]


<xarray.DataArray (lon: 200, v1: 40)>
array([[-0.41382027, -0.15785351,  0.82528279, ..., -0.30123779,
         0.22568314,  0.08955122],
       [ 0.30533284, -0.18584106, -0.28970698, ...,  0.42264679,
        -0.30883199,  0.0875411 ],
       [ 0.23671909,  0.38510944,  0.33949475, ...,  0.25012694,
        -0.77805428, -0.37047683],
       ...,
       [-0.3513459 , -0.31801708,  0.10346354, ..., -0.23538922,
        -0.21909046, -0.19984967],
       [-0.26693837,  0.18647294,  0.48971768, ..., -0.15780719,
         0.10123274,  0.11968855],
       [ 0.54642092,  0.65110925,  0.24353769, ..., -0.12211125,
        -0.20297169, -0.12063018]])
Coordinates:
  * lon      (lon) float32 40.0 40.05 40.1 40.15 40.2 ... 49.8 49.85 49.9 49.95
Dimensions without coordinates: v1

In [82]:
# create items
all_batches = list(map(lambda x: x, xrda_batches))
all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N", N=5), all_batches)) 
items_latent = list(itertools.chain(*[all_batches_latent]))

# CASE III - No Weight | No Label
dims_labels = None
weight = None
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels=dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(v1=0).data, xrda_batches.da)
rec_da

# CASE II - No Weight | Full Label
dims_labels = ["lat", "lon" , "z"]
weight = None
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da)
rec_da

# CASE I - Weight | Label
dims_labels = ["lat", "lon", "z"]
weight = np.ones([40,50])
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da)
rec_da

# CASE I - Weight | Half Labels
dims_labels = ["lat"]
weight = np.ones([40,])
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels, weight=weight)
rec_da

# CASE - No Weight | Mixed Labels
dims_labels = ["lon", "lat", "z"]
weight = None
all_batches_flipped = list(map(lambda x: repeat(x, "Lat Lon -> Lon Lat"), all_batches)) 
all_batches_latent_flipped = list(map(lambda x: repeat(x, "... -> ... N", N=5), all_batches_flipped)) 
items_latent_flipped = list(itertools.chain(*[all_batches_latent_flipped]))
rec_da = xrda_batches.reconstruct_from_items(items_latent_flipped, dims_labels=dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data.T, xrda_batches.da)
rec_da

# CASE I - Weight | Mixed Half Labels 
dims_labels = ["lon"]
weight = np.ones([50,])
all_batches_flipped = list(map(lambda x: repeat(x, "Lat Lon -> Lon Lat"), all_batches)) 
items_flipped = list(itertools.chain(*[all_batches_flipped]))
rec_da = xrda_batches.reconstruct_from_items(items_flipped, dims_labels, weight=weight)
rec_da



100%|██████████| 144/144 [00:00<00:00, 269.15it/s]


<xarray.DataArray (lon: 200, lat: 200, z: 5)>
array([[[-1.08563066, -1.08563066, -1.08563066, -1.08563066,
         -1.08563066],
        [ 0.70331013,  0.70331013,  0.70331013,  0.70331013,
          0.70331013],
        [ 1.53409028,  1.53409028,  1.53409028,  1.53409028,
          1.53409028],
        ...,
        [-1.25429702, -1.25429702, -1.25429702, -1.25429702,
         -1.25429702],
        [ 0.92227018,  0.92227018,  0.92227018,  0.92227018,
          0.92227018],
        [-0.03978635, -0.03978635, -0.03978635, -0.03978635,
         -0.03978635]],

       [[ 0.99734545,  0.99734545,  0.99734545,  0.99734545,
          0.99734545],
        [-0.59810531, -0.59810531, -0.59810531, -0.59810531,
         -0.59810531],
        [-0.52991408, -0.52991408, -0.52991408, -0.52991408,
         -0.52991408],
...
        [-0.13722242, -0.13722242, -0.13722242, -0.13722242,
         -0.13722242],
        [ 1.2875272 ,  1.2875272 ,  1.2875272 ,  1.2875272 ,
          1.2875272 ],
        [ 0.99480015,  0.99480015,  0.99480015,  0.99480015,
          0.99480015]],

       [[-0.21794626, -0.21794626, -0.21794626, -0.21794626,
         -0.21794626],
        [ 0.24402694,  0.24402694,  0.24402694,  0.24402694,
          0.24402694],
        [-1.27357018, -1.27357018, -1.27357018, -1.27357018,
         -1.27357018],
        ...,
        [-0.98969692, -0.98969692, -0.98969692, -0.98969692,
         -0.98969692],
        [ 0.52037948,  0.52037948,  0.52037948,  0.52037948,
          0.52037948],
        [-0.80351275, -0.80351275, -0.80351275, -0.80351275,
         -0.80351275]]])
Coordinates:
  * lat      (lat) float32 -60.0 -59.95 -59.9 -59.85 ... -50.15 -50.1 -50.05
  * lon      (lon) float32 40.0 40.05 40.1 40.15 40.2 ... 49.8 49.85 49.9 49.95
Dimensions without coordinates: z

In [ ]:
from einops import rearrange
all_batches = list(map(lambda x: x, xrda_batches))
all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N", N=5), all_batches)) 
items_latent = list(itertools.chain(*[all_batches_latent]))

# CASE I - Weight  | Label (All)
dims_labels_latent = ["x", "z"]
weight = np.ones([30,])
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da)

# CASE I - Weight  | Label (All) - switched
dims_labels_latent = ["z", "x"]
weight = np.ones([30,])
all_batches_latent_flipped = list(map(lambda x: repeat(x, "D Z -> Z D"), all_batches_latent)) 
items_latent_flipped = list(itertools.chain(*[all_batches_latent_flipped]))
rec_da = xrda_batches.reconstruct_from_items(items_latent_flipped, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da)

# CASE I -  No Weight |  Label (All) - Switched
dims_labels_latent = ["z", "x"]
weight = None
all_batches_latent_flipped = list(map(lambda x: repeat(x, "D Z -> Z D"), all_batches_latent)) 
items_latent_flipped = list(itertools.chain(*[all_batches_latent_flipped]))
rec_da = xrda_batches.reconstruct_from_items(items_latent_flipped, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da)


# CASE I -  No Weight | Half Label (All)
dims_labels_latent = ["x"]
weight = None
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(v1=0).data, xrda_batches.da)

# CASE I -  No Weight | No Label (All)
dims_labels_latent = None
weight = None
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(v1=0).data, xrda_batches.da)

rec_da

100%|██████████| 67/67 [00:00<00:00, 440.82it/s]


<xarray.DataArray (x: 360, v1: 5)>
array([[-1.08563066, -1.08563066, -1.08563066, -1.08563066, -1.08563066],
       [ 0.99734545,  0.99734545,  0.99734545,  0.99734545,  0.99734545],
       [ 0.2829785 ,  0.2829785 ,  0.2829785 ,  0.2829785 ,  0.2829785 ],
       ...,
       [ 2.05646729,  2.05646729,  2.05646729,  2.05646729,  2.05646729],
       [-1.88849235, -1.88849235, -1.88849235, -1.88849235, -1.88849235],
       [-1.12833059, -1.12833059, -1.12833059, -1.12833059, -1.12833059]])
Coordinates:
  * x        (x) object 1 2 3 4 5 6 7 8 9 ... 353 354 355 356 357 358 359 360
Dimensions without coordinates: v1

In [18]:
all_coords[0][["lon"]].coords

Coordinates:
  * lon      (lon) float32 40.0 40.05 40.1 40.15 40.2 ... 42.3 42.35 42.4 42.45

In [258]:
# reconstruct_from_items(items_latent, ["x","z"], coords)
w = np.ones((40,200))
w = 10*np.ones((40,50))
dims_label = ["lat", "y"]
items = list(itertools.chain(*[all_batches]))
# rec_da = xrda_batches.reconstruct(all_batches, dims_label, weight=w
rec_da = xrda_batches.reconstruct([all_batches], dims_label, weight=w)
# rec_da = reconstruct_from_items(items_latent, dims_label_latent, ds, weight=w)

# np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)


100%|████████████████████████████████████████| 144/144 [00:00<00:00, 540.49it/s]


In [265]:
list(rec_da.coords.keys())
# np.testing.assert_array_equal(rec_da["lat"], xrda_batches.da["lat"])

['lat']

In [223]:
rec_da.coords.dims

('lat', 'v1')

In [205]:
rec_da.min(), rec_da.max()

(<xarray.DataArray ()>
 array(1.),
 <xarray.DataArray ()>
 array(1.))

In [55]:
from oceanbench._src.datasets.utils import reconstruct_from_items

# reconstruct_from_items(items_latent, ["x","z"], coords)
w = np.ones((40,200))
dims_label_latent = ["lat", "lon", "z"]

all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N", N=5), all_batches)) 
items_latent = list(itertools.chain(*[all_batches_latent]))

rec_da = reconstruct_from_items(items_latent, dims_label_latent, xrda_batches, weight=w)

np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da.data)

100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 185.41it/s]


## Test Case III - 2D+T Field

In [38]:
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, 
    TimeAxis, SSH2D, SSH2DT
)

In [39]:
lon_axis = LongitudeAxis.init_from_limits(lon_min=40, lon_max=50-0.05, dlon=0.05)
lat_axis = LatitudeAxis.init_from_limits(lat_min=-60, lat_max=-50-0.05, dlat=0.05)
time_axis = TimeAxis.init_from_limits("2012-01-01", "2012-01-30", "1D")

da = SSH2DT.init_from_axis(lon=lon_axis, lat=lat_axis, time=time_axis)
da = asdataarray(da).to_dataset()
da

<xarray.Dataset>
Dimensions:  (time: 30, lat: 200, lon: 200)
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 2012-01-02 ... 2012-01-30
  * lat      (lat) float32 -60.0 -59.95 -59.9 -59.85 ... -50.15 -50.1 -50.05
  * lon      (lon) float32 40.0 40.05 40.1 40.15 40.2 ... 49.8 49.85 49.9 49.95
Data variables:
    ssh      (time, lat, lon) float32 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0

In [40]:
# patches = {"time": 10, "lat": 40, "lon": 20, }
patches = {"lon": 20, "time": 10, "lat": 40, }
strides = {"time": 10, "lat": 40, "lon": 20, }
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da.ssh,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

# assert ds[0].shape == (len(var.data),)
# np.testing.assert_array_equal(ds[0], np.arange(0, len(var.x), 1))
print(xrda_batches)
print(f"Torch Dataset(size): {len(xrda_batches)}")
# print(f"Batch (size): {ds[0].shape}")

XArray Patcher
DataArray size: OrderedDict([('time', 30), ('lat', 200), ('lon', 200)])
Patches:        OrderedDict([('time', 10), ('lat', 40), ('lon', 20)])
Strides:        OrderedDict([('time', 10), ('lat', 40), ('lon', 20)])
Num Batches:    OrderedDict([('time', 3), ('lat', 5), ('lon', 10)])
Torch Dataset(size): 150


In [41]:
import functools as ft
import itertools as it
from einops import repeat

all_batches = list(map(lambda x: x, xrda_batches))


all_coords = xrda_batches.get_coords()

assert len(all_batches) == len(all_coords)
all_batches[0].shape

(10, 40, 20)

In [42]:
from oceanbench._src.datasets.utils import reconstruct_from_items

# reconstruct_from_items(items_latent, ["x","z"], coords)
w = np.ones((10,40,20))
dims_label = ["time", "lat", "lon"]
items = list(itertools.chain(*[all_batches]))
# rec_da = xrda_batches.reconstruct(all_batches, dims_label, weight=w)
rec_da = reconstruct_from_items(items, dims_label, xrda_batches, weight=w)
# rec_da = reconstruct_from_items(items_latent, dims_label_latent, ds, weight=w)

np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)

100%|████████████████████████████████████████| 150/150 [00:00<00:00, 220.63it/s]


In [43]:
from oceanbench._src.datasets.utils import reconstruct_from_items

w = np.ones((10,40,20))
dims_label_latent = ["time", "lat", "lon", "z", "d"]

all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N M", N=5, M=10), all_batches)) 
items_latent = list(itertools.chain(*[all_batches_latent]))

rec_da = reconstruct_from_items(items_latent, dims_label_latent, xrda_batches, weight=w)

100%|█████████████████████████████████████████| 150/150 [00:03<00:00, 43.59it/s]


In [44]:
np.testing.assert_array_almost_equal(rec_da.isel(z=0, d=0).data, xrda_batches.da.data)

In [45]:
rec_da

<xarray.DataArray (time: 30, lat: 200, lon: 200, z: 5, d: 10)>
array([[[[[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]],

         [[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]],

         [[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]],

         ...,

...

         ...,

         [[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]],

         [[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]],

         [[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]]]]])
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 2012-01-02 ... 2012-01-30
  * lat      (lat) float32 -60.0 -59.95 -59.9 -59.85 ... -50.15 -50.1 -50.05
  * lon      (lon) float32 40.0 40.05 40.1 40.15 40.2 ... 49.8 49.85 49.9 49.95
Dimensions without coordinates: z, d